# Dene Stalk 

#### NLP soft weighted voting ensemble multi-class classifier to predict spend categories using Government of California’s purchase order data

BrianStation Data Science Diploma Capstone, March 2020

In [703]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

The data set used was the [Goverment of California's 2012-2015 Purchase Orders ](https://data.ca.gov/dataset/purchase-order-data/resource/bb82edc5-9c78-44e2-8947-68ece26197c5) that are mapped the United Nations Standard Products & Services Code (UNSPSC). A Group Title column was added using excel (vlookup) for Goods and Services based on Segment Title. 
 

<img src="UNSPSC.PNG"
     style="float: left; margin-right: 10px;" />

In [704]:
df_original = pd.read_csv('https://data.ca.gov/dataset/ae343670-f827-4bc8-9d44-2af937d60190/resource/bb82edc5-9c78-44e2-8947-68ece26197c5/download/purchase-order-data-2012-2015-.csv')
df = df_original.copy() 

## Initial Exploration

The dataset is made up of 344,504 rows (line items) with 32 columns comprised of 9 float and 23 object data types. The dataset has numerous columns specific to the Government of California (e.g. Acquisition Method), however the goal of this project is to develop a model that is transferable to other organizations and are disgarded.

In [705]:
# check dataset shape
df.shape

(344504, 32)

In [706]:
# peak into the dataset
df.sample(5)

,Creation Date,Purchase Date,Fiscal Year,LPA Number,Purchase Order Number,Requisition Number,Acquisition Type,Sub-Acquisition Type,Acquisition Method,Sub-Acquisition Method,...,Normalized UNSPSC,Commodity Title,Class,Class Title,Family,Family Title,Segment,Segment Title,Location,REMOVE AMERISOURCE
333189,5/22/2014,5/22/2014,2013-2014,NaN,C0000416,NaN,NON-IT Goods,NaN,Formal Competitive,NaN,...,45111602.0,Projection lamps,45111600.0,Projectors and supplies,45110000.0,Audio and visual presentation and composing eq...,45000000.0,Printing and Photographic and Audio and Visual...,"92105\n(32.738754, -117.094021)",NaN
158155,12/30/2014,12/24/2014,2014-2015,NaN,4500218500,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,27113201.0,General tool kits,27113200.0,Tool kits,27110000.0,Hand tools,27000000.0,Tools and General Machinery,"95691\n(38.575311, -121.560401)",NaN
233800,12/10/2012,12/10/2012,2012-2013,NaN,4500162549,NaN,NON-IT Goods,NaN,Services are specifically exempt by statute,NaN,...,50131606.0,Fresh eggs,50131600.0,Eggs and egg substitutes,50130000.0,Dairy products and eggs,50000000.0,Food Beverage and Tobacco Products,"73529\n(34.361458, -97.971748)",NaN
296113,5/22/2013,2/4/2013,2012-2013,NaN,AF54738,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,23301501.0,Wire and cable cutting and terminal assembly e...,23301500.0,Wire working machinery and equipment and acces...,23300000.0,Wire machinery and equipment,23000000.0,Industrial Manufacturing and Processing Machin...,"91609\n(34.16581, -118.378967)",NaN
58984,7/26/2013,11/2/2010,2013-2014,NaN,CFF 7228,NaN,NON-IT Services,Personal Services,Formal Competitive,NaN,...,55101515.0,Promotional material or annual reports,55101500.0,Printed publications,55100000.0,Printed media,55000000.0,Published Products,"90301\n(33.956998, -118.358491)",NaN


There are five main themes of the columns in the original dataset. However only Item Name, Item Description and Supplier Name were used as features with Segment Title as the target. Quantity and Unit Price and Total Price were also in the final dataset but not incorporated in the modelling process, however are used in interpreting the financial impact of the predictions. Below are some of the columns.

Dates:
+ Creation Date
- Purchase Date
- Fiscal Year

Internal Information:
* Department
* Acquisition Type
* Acquisition Method

Supplier Information:
* **Supplier Name**
* Qualifications
* Location

Purchase Information:
* PO number
* *Quantity*
* *Unit price*
* *Total price*
* **Item Name**
* **Item Description**

UNSPSC:
* **Segment Title** and corresponding code
* Family Title and corresponding code
* Class Title and corresponding code
* Commodity Title and corresponding code

In [707]:
# explore object columns
df.describe(include=['object']).transpose()

,count,unique,top,freq
Creation Date,344504,1015,6/27/2014,1531
Purchase Date,327083,2266,7/1/2014,4433
Fiscal Year,344504,3,2013-2014,120158
LPA Number,90897,1418,7-11-51-02,9267
Purchase Order Number,344504,197000,4500211314,602
Requisition Number,14366,5996,REQ0008872,123
Acquisition Type,344504,5,NON-IT Goods,213578
Sub-Acquisition Type,68334,25,Personal Services,16104
Acquisition Method,344504,20,Informal Competitive,82046
Sub-Acquisition Method,30883,16,Fleet,14148


In [708]:
# explore numeric columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Supplier Code,344468.0,8.426214e+05,2.466547e+06,0.000000e+00,26819.0,1012554.0,1493440.0,954831781.0
Quantity,344474.0,1.945776e+03,9.819831e+04,1.000000e-04,1.0,1.0,6.0,20000000.0
Normalized UNSPSC,343487.0,4.926200e+07,2.248214e+07,3.018170e+05,39111610.0,44121506.0,71151106.0,95141903.0
Class,341211.0,4.921572e+07,2.247339e+07,1.010150e+07,39111500.0,44121500.0,71151000.0,95141900.0
Family,341211.0,4.921157e+07,2.247664e+07,3.018000e+06,39110000.0,44120000.0,71150000.0,95140000.0
Segment,341211.0,4.907212e+07,2.247862e+07,1.000000e+07,39000000.0,44000000.0,71000000.0,95000000.0
REMOVE AMERISOURCE,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Kept only relevant columns to reduce dataset and computational needs for cleaning. While the model only incorporates Supplier Name, Item Name and Item Description as features and Segment as target, kept Quantity, Unit Price and Total price for future analysis and cleaning. 

In [709]:
# only keep relevant columns going forward
df = df[['Quantity', 'Unit Price', 'Total Price', 'Supplier Name', 'Item Name', 'Item Description', 'Segment Title']]

## Cleaning Process

As can be seen in the info below, Unit Price and Total Price are objects. They also contain punctuation (e.g. $) that needs to be removed before being converted to a float.

In [710]:
# check column data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344504 entries, 0 to 344503
Data columns (total 7 columns):
Quantity            344474 non-null float64
Unit Price          344474 non-null object
Total Price         344474 non-null object
Supplier Name       344468 non-null object
Item Name           344473 non-null object
Item Description    344305 non-null object
Segment Title       341211 non-null object
dtypes: float64(1), object(6)
memory usage: 18.4+ MB


In [711]:
# example of formatting 
df[['Unit Price', 'Total Price']].sample(5)

,Unit Price,Total Price
256418,"$20,727.25","$20,727.25"
334861,"$5,440.00","$5,440.00"
61932,$68.00,$816.00
230572,"$29,172.00","$29,172.00"
68577,$22.00,"$1,430.00"


Remove punctuation and blank spaces before converting Unit Price and Total Price to a float. However, the dataset uses the accounting formatting for currency with () around negative values. The below cleaning takes that into account by changing the first parenthesis to a negative sign. 

In [712]:
import string

# create punctuation list
punctuation_list = list(string.punctuation)

# remove negative sign and first paranthesis for allow for correct label of negative amounts
punctuation_list.remove('-')
punctuation_list.remove('(')

# add blank spaces
punctuation_list.append(" ")

punctuation_list = ''.join(punctuation_list)

neg_conversion = str.maketrans('(', '-', punctuation_list)
#tbl = str.maketrans('(', '-', '$)')


df['Total Price'] = df['Total Price'].astype(str).apply(lambda x: x.translate(neg_conversion))
df['Unit Price'] = df['Unit Price'].astype(str).apply(lambda x: x.translate(neg_conversion))

df[['Total Price', 'Unit Price']] = df[['Total Price', 'Unit Price']].astype(float)

# check to ensure changes were applied correctly
print('Total Price dtype after cleaning: ', df['Total Price'].dtype)
print('Unit Price dtype after cleaning: ', df['Unit Price'].dtype)

Total Price dtype after cleaning:  float64
Unit Price dtype after cleaning:  float64


In [713]:
df[['Unit Price', 'Total Price']].sum()

Unit Price     1.491027e+13
Total Price    1.505179e+13
dtype: float64

In [714]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,344474.0,1.945776e+03,9.819831e+04,1.000000e-04,1.00,1.0,6.00,2.000000e+07
Unit Price,344474.0,4.328418e+07,2.141131e+09,-3.086123e+09,3427.25,54000.0,1000054.25,7.337038e+11
Total Price,344474.0,4.369499e+07,2.141135e+09,-3.086123e+09,30000.00,352539.0,1461666.50,7.337038e+11


While the raw dataset was relatively clean, some additional cleaning was required. This included removing rows where:
* Segment Title was blank
* Total Price was <= 0. Refunds and interdepartmental reimbursements are out of the scope of this project.
* Supplier Name was input as 'Unknown' or blank
* Item Name was blank
* Item Description was input as 'confidential' or blank

Below is the count of the number of rows removed based on the above conditions

In [715]:
print('Count of rows with Segment Title as blank: ', df['Segment Title'].isnull().sum())
print('Count of rows with Total Price less than or equal to $0: ', df['Total Price'].le(0).sum())
print('Count of rows with Supplier Name as "Unknown": ', (df['Supplier Name'] == 'Unknown').sum())
print('Count of rows with Supplier Name as blank: ', df['Supplier Name'].isnull().sum())
print('Count of rows with Item Name as blank: ', df['Item Name'].isnull().sum())
print('Count of rows with Item Description as "confidential": ', (df['Item Description'] == 'confidential').sum())
print('Count of rows with Item Description as blank: ', df['Item Description'].isnull().sum())

Count of rows with Segment Title as blank:  3293
Count of rows with Total Price less than or equal to $0:  8949
Count of rows with Supplier Name as "Unknown":  4473
Count of rows with Supplier Name as blank:  36
Count of rows with Item Name as blank:  31
Count of rows with Item Description as "confidential":  1637
Count of rows with Item Description as blank:  199


In [716]:
# only keep rows with Segment Title inputs

df = df[df['Segment Title'].notna()]

# drop rows with negative or 0 spends
df = df.drop(df[df['Total Price'] <= 0].index)

# drop suppliers listed as 'unknown'
df = df.drop(df[df['Supplier Name'] == 'Unknown'].index)

# drop rows with supplier name missing
df = df.drop(df[df['Supplier Name'].isna() == True].index)

# drop rows with item name missing
df = df.drop(df[df['Item Name'].isna() == True].index)

# drop rows with item description as 'confidential'
df = df.drop(df[df['Item Description'] == 'confidential'].index)

# drop rows with item description missing
df = df.drop(df[df['Item Description'].isna() == True].index)

# check new df shape
print('New dateframe shape: ', df.shape)

New dateframe shape:  (326308, 7)


In [717]:
# Change Segment Title to Segment Target
df = df.rename(columns={'Segment Title': 'Segment Target'})

## Feature Engineering

Segment Target was reduced from 55 to 26 Segment categories. This was accomplished by the bottom 20% of categories by total rows being grouped into either Other Goods or Other Services based on their commodity type. But first those categories needed to be found. A 47% reduction in spend categories resulting in only aggregating 7% of total spend. This was accomplished through the steps. 

First, a column identifying what categories are Services and which are Goods was created.

In [718]:
services_keywords_list = ['Services', 'services', 'Clubs']

services_keywords_list = '|'.join(services_keywords_list)

df['Services_dummy'] = pd.np.where(df['Segment Target'].str.contains(services_keywords_list, case=True),1,0)

df.sample(5)

,Quantity,Unit Price,Total Price,Supplier Name,Item Name,Item Description,Segment Target,Services_dummy
271569,6.0,19900.0,119400.0,THE PRIMARY SOURCE,Printer kits,Printer kits,Office Equipment and Accessories and Supplies,0
99269,1.0,4832469.0,4832469.0,Basalite Concrete Products LLC,Cement,Cement,Structures and Building and Construction and M...,0
256077,1.0,4523554.0,4523554.0,MJ INDUSTRIAL SUPPLIES,Water Heater,X,Distribution and Conditioning Systems and Equi...,0
196347,6.0,10168.0,61008.0,Grainger Industrial Supply,Floor Mats,Floor Mats,Defense and Law Enforcement and Security and S...,0
49710,400.0,3975.0,1590000.0,"e Poly Star, Inc",33x48 orange plastic bags,33x48 orange plastic bags,Material Handling and Conditioning and Storage...,0


Create a dataframe to check categories where classified appropriately

In [719]:
# create dataframe with only unique values
df_services= df[['Segment Target', 'Services_dummy']].drop_duplicates()

# sort table
df_services = df_services.sort_values(['Services_dummy', 'Segment Target'], ascending=[False,True]) 

# display table with a widened columns for Segment
df_services.style.set_properties(subset=['Segment Target'], **{'width': '500px'})

,Segment Target,Services_dummy
1788,Building and Facility Construction and Maintenance Services,1
37,Editorial and Design and Graphic and Fine Art Services,1
895,Education and Training Services,1
67,Engineering and Research and Technology Based Services,1
878,Environmental Services,1
932,Farming and Fishing and Forestry and Wildlife Contracting Services,1
1573,Financial and Insurance Services,1
523,Healthcare Services,1
87,Industrial Cleaning Services,1
1171,Industrial Production and Manufacturing Services,1


Create a dataframe to identify the categories in the bottom 20% of line item frequency

In [720]:
# create a dataframe with Segment, Services Total Price grouped by Segment with sum and count (number of rows) of Total Price 
segment_target_df = pd.DataFrame(df.groupby(['Segment Target','Services_dummy'])['Total Price'].agg(['sum','count']).reset_index())

# sort by count
segment_target_df= segment_target_df.sort_values(by=('count'), ascending = False)

# create a column for cummaltive amount of rows
segment_target_df['cum_sum_count'] = segment_target_df['count'].cumsum()

# create a column for cummaltive percent of rows
segment_target_df['cum_perc_count'] = round(100*segment_target_df['cum_sum_count']/segment_target_df['count'].sum(),1)

# create a column for cummaltive sum of total spend
segment_target_df['cum_sum_sum'] = segment_target_df['sum'].cumsum()

# create a column for cummaltive percent of total spend
segment_target_df['cum_perc_sum'] = round(100*segment_target_df['cum_sum_sum']/segment_target_df['sum'].sum(),1)

Using a nested where clause for segements with the grouping of bottom 20% of segments into either Other Goods or Other Services based on commmodity type

In [721]:
# using a nested where clause for segements with the grouping of bottom 20% of segments into either Other Goods or Other Services based on commmodity type
segment_target_df['Reduced Segment Target'] = np.where((segment_target_df['cum_perc_count']>80.5) & (segment_target_df['Services_dummy'] == 1), 'Other Services', \
                                                       np.where((segment_target_df['cum_perc_count']>80.5) & (segment_target_df['Services_dummy'] == 0), 'Other Goods', segment_target_df['Segment Target']))

Create numeric categories of the final string category names to be used in the modelling portion of this project. 

In [722]:
# factorize reduced segement target for modelling 
segment_target_df['Segment Target Cat'] = segment_target_df['Reduced Segment Target'].factorize()[0]

# convert new column is dtype string instead of integer 
segment_target_df['Segment Target Cat'] = segment_target_df['Segment Target Cat'].astype(str) 


# display table with a widened columns for Segment 
segment_target_df.style.set_properties(subset=['Segment Target', 'Reduced Segment Target'], **{'width': '300px'})

,Segment Target,Services_dummy,sum,count,cum_sum_count,cum_perc_count,cum_sum_sum,cum_perc_sum,Reduced Segment Target,Segment Target Cat
26,Information Technology Broadcasting and Telecommunications,0,1.92812e+11,30275,30275,9.3,1.92812e+11,1.3,Information Technology Broadcasting and Telecommunications,0
19,Food Beverage and Tobacco Products,0,4.66071e+10,27682,57957,17.8,2.39419e+11,1.6,Food Beverage and Tobacco Products,1
39,Office Equipment and Accessories and Supplies,0,1.2509e+10,27007,84964,26,2.51928e+11,1.7,Office Equipment and Accessories and Supplies,2
14,Engineering and Research and Technology Based Services,1,4.27013e+11,16589,101553,31.1,6.78942e+11,4.6,Engineering and Research and Technology Based Services,3
33,Medical Equipment and Accessories and Supplies,0,2.99421e+10,16231,117784,36.1,7.08884e+11,4.8,Medical Equipment and Accessories and Supplies,4
20,Fuels and Fuel Additives and Lubricants and Anti corrosive Materials,0,3.22231e+10,15989,133773,41,7.41107e+11,5,Fuels and Fuel Additives and Lubricants and Anti corrosive Materials,5
41,Paper Materials and Products,0,1.33684e+10,11054,144827,44.4,7.54475e+11,5.1,Paper Materials and Products,6
5,Commercial and Military and Private Vehicles and their Accessories and Components,0,1.09756e+11,10314,155141,47.5,8.64231e+11,5.9,Commercial and Military and Private Vehicles and their Accessories and Components,7
11,Education and Training Services,1,3.28965e+11,9959,165100,50.6,1.1932e+12,8.1,Education and Training Services,8
6,Defense and Law Enforcement and Security and Safety Equipment and Supplies,0,3.40619e+10,9943,175043,53.6,1.22726e+12,8.3,Defense and Law Enforcement and Security and Safety Equipment and Supplies,9


Merge cleaned df and newly created segment_target_df

In [723]:
# left join on Segment Target
df_merged = pd.merge(df,segment_target_df[['Segment Target','Reduced Segment Target', 'Segment Target Cat']], how='left', left_on='Segment Target', right_on ='Segment Target')


Since the original dataset did not have goods and services, that column was dropped. 

In [724]:
# drop services_dummy and original Segment Target
df_merged = df_merged.drop(['Services_dummy', 'Segment Target'], axis=1)

In [725]:
# rename reduced segment target to segment target for the modelling portion
df_merged = df_merged.rename(columns={'Reduced Segment Target': 'Segment Target'})

## Finalized Cleaned Dataset

Check to see if there are any nulls, the correct columns are present in the correct data type.  
  
The cleaned dataset has 326,208 rows with 8 columns, 3 float and 5 object.

In [726]:
print(df_merged.shape)

(326308, 8)


In [727]:
# check info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326308 entries, 17 to 344503
Data columns (total 8 columns):
Quantity            326308 non-null float64
Unit Price          326308 non-null float64
Total Price         326308 non-null float64
Supplier Name       326308 non-null object
Item Name           326308 non-null object
Item Description    326308 non-null object
Segment Target      326308 non-null object
Services_dummy      326308 non-null int32
dtypes: float64(3), int32(1), object(4)
memory usage: 31.2+ MB


In [728]:
# check to ensure there are no null values
df_merged.isnull().sum()

Quantity              0
Unit Price            0
Total Price           0
Supplier Name         0
Item Name             0
Item Description      0
Segment Target        0
Segment Target Cat    0
dtype: int64

In [729]:
# check object statistics
df_merged.describe(include=['object']).transpose()

,count,unique,top,freq
Supplier Name,326308,24430,Voyager Fleet Systems Inc,13756
Item Name,326308,170898,Medical Supplies,2882
Item Description,326308,207616,Medical Supplies,1361
Segment Target,326308,26,Other Goods,49833
Segment Target Cat,326308,26,24,49833


In [730]:
# check float statisitcs
df_merged.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,326308.0,2.043375e+03,1.008739e+05,0.004,1.0,1.0,7.0,2.000000e+07
Unit Price,326308.0,4.475731e+07,2.183321e+09,0.000,4100.0,60000.0,1018935.0,7.337038e+11
Total Price,326308.0,4.517937e+07,2.183325e+09,1.000,35200.0,385195.5,1488000.0,7.337038e+11


In [733]:
# one last check 
df_merged.sample(10)

,Quantity,Unit Price,Total Price,Supplier Name,Item Name,Item Description,Segment Target,Segment Target Cat
210246,400.0,3006.0,1202400.0,STAY SAFE SUPPLY,"PAPER,COPY,20 LB,8-1/2"" X 11"",WHT","PAPER,COPY,20 LB,8-1/2"" X 11"",WHT",Paper Materials and Products,6
180501,3.0,3000.0,9000.0,Prison Industry Authority,Set-in-place,Set-in-place,Industrial Cleaning Services,14
215866,1.0,90000000.0,90000000.0,ORRICK HERRINGTON & SUTCLIFFE,AM-07 Increase time and money.,This amendment extends the termination date by...,Other Services,25
200951,2.0,8374000.0,16748000.0,Xerox Corporation,Equipment and software maintenance.,Xerox printer equipment and software maintenance,Engineering and Research and Technology Based ...,3
290191,5.0,138054.0,690270.0,Fastenal,"HERBICIDE,ESPLANADE,200SC","HERBICIDE,ESPLANADE,200SC",Other Goods,24
156271,1.0,1070000.0,1070000.0,RIVCOMM LLC,HAVIS KK12001 PRISONER TRANSPORT INSERT,HAVIS KK12001 PRISONER TRANSPORT INSERT,Other Goods,24
80943,72.0,1195.0,86040.0,"International Restaurant Equipment Co., Inc.",STEAMTABLE PANS ONE-FOURTH LIDS S/S,STEAMTABLE PANS ONE-FOURTH LIDS S/S,Other Goods,24
285273,1.0,311950.0,311950.0,Future Data Systems,keyboard,keyboard,Information Technology Broadcasting and Teleco...,0
99302,4.0,1500.0,6000.0,"C-Tech Associates, Inc.",coupler,coupler,Furniture and Furnishings,10
254494,1.0,939600.0,939600.0,Sacramento Marriott,LODGING,TO PROVIDE LODGING FOR FIRE PERSONNEL ON SAND ...,Other Services,25


In [732]:
# save cleaned data to be used for modelling (see included notebook Modelling)
df.to_csv('GoC Spend Data Cleaned (Quantity, Unit Price, Total Price, Supplier Name, Item Name, Item Description, UNSPSC Segment).csv', index=False)

<img src="data.PNG"
     style="float: left; margin-right: 10px;" />

#### Modelling process can be found in the Modelling notebook